# Основная информация проекта

## Цель проекта
Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

## Описание датасета
1. **school** - аббревиатура школы, в которой учится ученик;
2. **sex** - пол ученика ('F' - женский, 'M' - мужской);
3. **age** - возраст ученика (от 15 до 22);
4. **address** - тип адреса ученика ('U' - городской, 'R' - за городом);
5. **famsize** - размер семьи('LE3' <= 3, 'GT3' >3);
6. **Pstatus** - статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно);
7. **Medu** - образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее);
8. **Fedu** - образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее);
9. **Mjob** - работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое);
10. **Fjob** - работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое);
11. **reason** — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое);
12. **guardian** - опекун ('mother' - мать, 'father' - отец, 'other' - другое);
13. **traveltime** - время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.);
14. **studytime** - время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов);
15. **failures** - количество внеучебных неудач (n, если 1<=n<=3, иначе 0);
16. **schoolsup** - дополнительная образовательная поддержка (yes или no);
17. **famsup** - семейная образовательная поддержка (yes или no);
18. **paid** - дополнительные платные занятия по математике (yes или no);
19. **activities** - дополнительные внеучебные занятия (yes или no);
20. **nursery** - посещал детский сад (yes или no);
21. **higher** - хочет получить высшее образование (yes или no);
22. **internet** - наличие интернета дома (yes или no);
23. **romantic** - в романтических отношениях (yes или no);
24. **famrel** - семейные отношения (от 1 - очень плохо до 5 - очень хорошо);
25. **freetime** - свободное время после школы (от 1 - очень мало до 5 - очень мого);
26. **goout** - проведение времени с друзьями (от 1 - очень мало до 5 - очень много);
27. **health** - текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо);
28. **absences** - количество пропущенных занятий;
29. **score** - баллы по госэкзамену по математике.

# Импорт библиотек и загрузка данных

In [5]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Установка максимального количества строк для отображения
pd.set_option('display.max_rows', 50)
# Установка максимального количества стоолбцов для отображения
pd.set_option('display.max_columns', 50)

In [7]:
stud_eda = pd.read_csv('stud_math.csv') # Загрузка датасета

# Функции для обработки и анализа данных

In [140]:
def parsing_all_columns_data(data):
    '''Функция для анализа всех столбцов на количество 
    уникальных значений и количество пропусков'''
    for col in data.columns:
        unique_values = len(data[col].value_counts())
        empty_values = data[col].isna().sum()
        print(
            f"В столбце '{col}' уникальных значений - {unique_values} ,а количество пропусков - {empty_values}.")

In [19]:
def cols_to_drop(data):
    '''Функция для определия столбцов, 
    не влияющих на предсказываемую величину'''
    # Поиск колонок с большим количеством пропусков
    many_null_cols = [
        col for col in data.columns if data[col].isnull().sum() / data.shape[0] > 0.9]
    # Поиск колонок в которых одно из значений встречается > 90%
    big_top_value_cols = [col for col in data.columns if data[col].value_counts(
        dropna=False, normalize=True).values[0] > 0.9]
    # Поиск колонок в которых только одно значение
    one_value_cols = [col for col in data.columns if data[col].nunique() <= 1]
    return list(set(many_null_cols + big_top_value_cols + one_value_cols))

# Обзор данных

In [146]:
# Проверка на успешность загрузки данных и краткий вывод таблицы с информацией
display(stud_eda.head(10))
stud_eda.info()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,3.0,yes,no,NaN,no,yes,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,home,mother,1.0,3.0,0.0,no,yes,yes,yes,yes,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
5,GP,M,16,U,LE3,T,4.0,3.0,services,other,reputation,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,4.0,2.0,5.0,10.0,75.0
6,GP,M,16,NaN,LE3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,no,no,yes,-6.0,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,home,mother,2.0,2.0,0.0,yes,yes,no,no,yes,-6.0,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
8,GP,M,15,U,LE3,A,3.0,2.0,services,other,home,mother,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,yes,no,NaN,2.0,2.0,1.0,0.0,95.0
9,GP,M,15,U,NaN,NaN,3.0,4.0,other,other,home,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,5.0,1.0,5.0,0.0,75.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

In [135]:
# Вывод количества строк и столбцов таблицы
print(f'Датасет состоит из {stud_eda.shape[0]} строк и {stud_eda.shape[1]} столбцов')

Датасет состоит из 395 строк и 30 столбцов


In [136]:
# Подсчёт уникальных значений и количество пропусков в столбцак датасета
parsing_all_columns_data(stud_eda)

В столбце 'school' уникальных значений - 2 ,а количество пропусков - 0.
В столбце 'sex' уникальных значений - 2 ,а количество пропусков - 0.
В столбце 'age' уникальных значений - 8 ,а количество пропусков - 0.
В столбце 'address' уникальных значений - 2 ,а количество пропусков - 17.
В столбце 'famsize' уникальных значений - 2 ,а количество пропусков - 27.
В столбце 'Pstatus' уникальных значений - 2 ,а количество пропусков - 45.
В столбце 'Medu' уникальных значений - 5 ,а количество пропусков - 3.
В столбце 'Fedu' уникальных значений - 6 ,а количество пропусков - 24.
В столбце 'Mjob' уникальных значений - 5 ,а количество пропусков - 19.
В столбце 'Fjob' уникальных значений - 5 ,а количество пропусков - 36.
В столбце 'reason' уникальных значений - 4 ,а количество пропусков - 17.
В столбце 'guardian' уникальных значений - 3 ,а количество пропусков - 31.
В столбце 'traveltime' уникальных значений - 4 ,а количество пропусков - 28.
В столбце 'studytime' уникальных значений - 4 ,а количество 

In [107]:
# Подсчёт количества столбцов с пропущенными значениями
print(
    f'Количество столбцов с пропущенными значениями - {data.isnull().any().sum()}')

Количество столбцов с пропущенными значениями - 27


# Prepare data

In [95]:
one = [col for col in data.columns if data[col].nunique() <= 1]
len(one)

0

In [85]:
many_null_cols = [col for col in student.columns if student[col].isnull(
).sum() / student.shape[0] > 0.9]
big_top_value_cols = [col for col in student.columns if student[col].value_counts(
    dropna=False, normalize=True).values[0] > 0.9]
one_value_cols = [
    col for col in student.columns if student[col].nunique() <= 1]

In [86]:
cols_to_drop = list(set(many_null_cols + big_top_value_cols + one_value_cols))
# cols_to_drop.remove('isFraud')
# len(cols_to_drop)
print(cols_to_drop)

['higher']


In [72]:
# test = student.drop(cols_to_drop, axis=1)
student.shape

(395, 30)

In [139]:
data.higher.value_counts()# display(data.higher.value_counts(dropna=False, normalize=True))

0.9012658227848102

In [9]:
 stud_eda['paid'].isnull().sum() # / 


40

In [11]:
stud_eda.shape[0]

395

In [14]:
360/395

0.9113924050632911